<img style="float: right;" src="images/cell2mol_logo.png" width="500">

# cell2mol
Unit Cell to Molecule Interpretation

In [1]:
import numpy as np
from ipywidgets import interactive, widgets, HTML, Button
from typing import Callable, Any
import warnings
import os
import markdown
import codecs
from IPython.display import display, FileLink
from contextlib import contextmanager
# Two imports are not needed due to visualization being disabled until nglview integration is fixed
#import ase.io
#import nglview


warnings.simplefilter('ignore')

In [2]:
html_description = markdown.markdown("""
cell2mol is able to analyze cif files containing crystallographic unit cell information and provide a chemical interpretation of the individual molecular species""")
description=HTML(html_description)

pre_accordion = widgets.Accordion()
pre_accordion.children = [description]
pre_accordion.set_title(0, 'Description')
pre_accordion.selected_index = None
pre_accordion

Accordion(children=(HTML(value='<p>cell2mol is able to analyze cif files containing crystallographic unit cell…

Proceed through the four tabs sequentially.

In [3]:
# Import the core functionalities of cell2mol
from cell2mol.cif2info import cif_2_info
from cell2mol.c2m_module import save_cell, cell2mol



In [4]:
# Define plotting and downloading functionalities

class DownloadButton(Button):
    """Download button with dynamic content

    The content is generated using a callback when the button is clicked.
    """

    def __init__(self, filename: str, contents: Callable[[], str], **kwargs):
        super(DownloadButton, self).__init__(**kwargs)
        self.filename = filename
        self.contents = contents
        self.tooltip='Click me to download'
        self.description='Download'
        self.icon='fa-check'
        self.on_click(self.__on_click)

    def __on_click(self, b):
        contents: bytes = self.contents().encode('utf-8')
        b64 = base64.b64encode(contents)
        payload = b64.decode()
        digest = hashlib.md5(contents).hexdigest()  # bypass browser cache
        id = f'dl_{digest}'

        display(HTML(f"""
<html>
<body>
<a id="{id}" download="{self.filename}" href="data:text/csv;base64,{payload}" download>
</a>

<script>
(function download() {{
document.getElementById('{id}').click();
}})()
</script>

</body>
</html>
"""))

class ExecutionButton(Button):
    def __init__(self, action: Callable[bool, Any], **kwargs):
        super(ExecutionButton, self).__init__(**kwargs)
        self.action = action
        self.tooltip='Click me to run'
        self.description='Run'
        self.icon='fa-check'
        self.on_click(self.__on_click)
        
    @contextmanager
    def show_loading(self):
        self.description = 'Running...'
        self.icon = 'fa-stop'
        yield
        self.description = 'Ready to run again'
        self.icon = 'fa-check'
        
    def __on_click(self, b):
        with self.show_loading():
            self.action(run=True)
              
def plot_from_file(filename):
    pass
    # This function is disabled currently due to incompatibilities depending on the nglviewer and widgets versions
    #atoms = ase.io.read(filename)
    #v = nglview.show_ase(self.conf)
    #v.add_representation("unitcell")
    #boxy=widgets.Box([v],layout=self.box_layout)
    #display(boxy)

In [5]:
#############################
# STEP 1: Choose and upload
#############################

# Define upload widget
Widget1UploadConfiguration = widgets.FileUpload(
 accept='.cif',  # Accepted file extension e.g. '.cif'
 multiple=False,  # True to accept multiple files upload else False
 wait=True
)

examples = ('Transition metal complex', 'cell2mol/test/cif/AJEPIP.cif'),
widget_choose_filename = widgets.Dropdown(options=examples,description='Structure:')

Widget1Out = widgets.Output()
def chooseAndPlotConfigurationAltOutput(filename):
    Widget1Out.clear_output()
    with Widget1Out:
        plot_from_file(filename)
        
Widget1ExampleConfiguration = interactive(chooseAndPlotConfigurationAltOutput, filename=widget_choose_filename)

# This function updates the dropdown list when a new file is uploaded
def updateWidget1ExampleConfiguration(*args):
    uploaded = next(iter(Widget1UploadConfiguration.value))
    cif_content = codecs.decode(uploaded['content'], encoding="utf-8") 
    cif_name = uploaded['name']
    mypath="./uploaded/"
    with open(mypath + cif_name, 'w+') as f: f.write(cif_content)
    found_files=[]
    for f in os.listdir(mypath):
        found_files.append(f)
    all_files = found_files
    widget_choose_filename.options = all_files
    
Widget1UploadConfiguration.observe(updateWidget1ExampleConfiguration,names='value')
Widget1ExamplesAndUpload = widgets.HBox([Widget1ExampleConfiguration,Widget1UploadConfiguration])
Widget1Text = HTML(markdown.markdown("""
Choose a structure from the examples or upload your own in .cif format [https://en.wikipedia.org/wiki/Crystallographic_Information_File].
"""))
Widget1 = widgets.VBox([Widget1Text,Widget1ExamplesAndUpload,Widget1Out])

#############################
# STEP 2: Run cell2info
#############################
def run_cell2info(run=False):
    if run :
        cif_name = widget_choose_filename.value
        if "/" in cif_name :
            cif_name = cif_name.split("/")[-1]
        refcode = cif_name.split("/")[-1].split(".")[-2]
        info_name = cif_name.replace('.cif', '.info')
        err_name = cif_name.replace('.cif', '.err')
        input_path = "./uploaded/" + cif_name
        info_path = "./uploaded/" + info_name
        error_path = "./uploaded/" + err_name
        cell_path = "./Cell_" + refcode +'.gmol'
        cif_2_info(input_path, info_path, error_path)

#############################
# STEP 3: Run cell2mol on infofile
#############################
def run_cell2mol(run=False):
    if run :
        cif_name = widget_choose_filename.value
        if "/" in cif_name :
            cif_name = cif_name.split("/")[-1]
        refcode = cif_name.split("/")[-1].split(".")[-2]
        info_name = cif_name.replace('.cif', '.info')
        err_name = cif_name.replace('.cif', '.err')
        input_path = "./uploaded/" + cif_name
        info_path = "./uploaded/" + info_name
        error_path = "./uploaded/" + err_name
        cell_path = "./Cell_" + refcode +'.gmol'
        if os.path.exists(info_path):
            print(f"Running in {os.getcwd()}")
            cell = cell2mol(info_path, refcode, os.getcwd(), 3)
        else:
            print("Please, wait until cell2info has finished for this input.")
    
Widget2Text = HTML(markdown.markdown("""
Will run cell2info on the cif file selected in the drop down menu in the previous tab. Should run almost instantaneously. When finished, go to next tab.
"""))
Widget2 = widgets.VBox([Widget2Text, ExecutionButton(run_cell2info)])
Widget3Text = HTML(markdown.markdown("""
Will run cell2mol on the info file obtained in the previous tab from the cif file in the first tab. Takes some time to run. When finished, go to next tab.
"""))
Widget3 = widgets.VBox([Widget3Text, ExecutionButton(run_cell2mol)])
Widget4 = DownloadButton("Cell","patatatatatat")

In [6]:
#############################
# Put widget in tab
#############################

children = [Widget1, Widget2, Widget3, Widget4]
tab = widgets.Tab() #layout=widgets.Layout(width='800px', height='800px'))
tab.children = children
tab.set_title(0, 'Upload file')
tab.set_title(1, 'Run cell2info')
tab.set_title(2, 'Run cell2mol')
tab.set_title(3, 'Download cell object')


In [7]:
display(tab)

In [8]:
html_acknowledgments = markdown.markdown("""
* Funding from the NCCR MARVEL funded by the SNSF.
""")
acknowledgments=widgets.VBox([
    HTML(html_acknowledgments)
])

html_howtocite = markdown.markdown("""
Please cite the cell2mol paper~!
""")
howtocite=widgets.VBox([
    HTML(html_howtocite)
])

post_children=[acknowledgments,howtocite]
post_accordion = widgets.Accordion(layout=widgets.Layout())
#print(post_accordion.layout.keys)
post_accordion.children = post_children
post_accordion.set_title(0, 'Acknowledgments')
post_accordion.set_title(1, 'How to cite')
post_accordion.selected_index = None
post_accordion

Accordion(children=(VBox(children=(HTML(value='<ul>\n<li>Funding from the NCCR MARVEL funded by the SNSF.</li>…

See the project on [GitHub](https://github.com/lcmd-epfl/cell2mol)

cell2mol, Copyright © 2022 LCMD-EPFL